### Build a Question Answering application over a Graph Database

You need to create a Neo4j database in Neo4j Aura https://console.neo4j.io
Then update the below connection parameters

In [1]:
NEO4J_URI="neo4j+s://c214b0b3.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""

In [2]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD
os.environ["allow_dangerous_code"]="True"

print(NEO4J_URI)
print(NEO4J_USERNAME)
print(NEO4J_PASSWORD)

neo4j+s://c214b0b3.databases.neo4j.io
neo4j
frZ6FD77u8TJCsCAolg1VzxO7rjLAfWKsGXkiTNbx2w


In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [4]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [5]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [6]:
graph.query(moview_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {title: STRING, id: STRING, released: DATE, imdbRating: FLOAT}
User {city: STRING, name: STRING, userId: INTEGER, age: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:POSTED]->(:Post)
(:User)-[:FRIEND]->(:User)
(:User)-[:LIKES]->(:User)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [9]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x764a6df180e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x764a6df18c20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
from langchain.chains import GraphCypherQAChain
from langchain.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema

# mychain=GraphCypherQAChain(allow_dangerous_requests=True, llm=llm, graph=graph, graph_schema)
# Cypher validation tool for relationship directions
#corrector_schema = [
#    Schema(el["start"], el["type"], el["end"])
#    for el in graph.structured_schema.get("relationships")
#]
#cypher_validation = CypherQueryCorrector(corrector_schema)

chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

ValueError: In order to use this chain, you must acknowledge that it can make dangerous requests by setting `allow_dangerous_requests` to `True`.You must narrowly scope the permissions of the database connection to only include necessary permissions. Failure to do so may result in data corruption or loss or reading sensitive data if such data is present in the database.Only use this chain if you understand the risks and have taken the necessary precautions. See https://python.langchain.com/docs/security for more information.

In [13]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Casino"})<-[:DIRECTED]-(p:Person)
RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'Martin Scorsese.  \n'}

In [14]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.  \n'}

In [17]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(p) AS artistCount
Full Context:
[{'artistCount': 1240}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1240 artists. \n'}

In [18]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN count(m) 

Full Context:
[{'count(m)': 3}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in', 'result': '3 \n'}